# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
val housing = spark.read.format("csv").option("header","true").option("inferSchema","true").load("data/housing.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-QRC7UDIG-di-Riccardo:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1574329269254)
SparkSession available as 'spark'


housing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 8 more fields]


---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [2]:
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [3]:
housing.count()

res1: Long = 20640


## 2.2. Look at the data
Print the first five records of the dataset.

In [4]:
housing.select("*").show(5,false)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|-122.23  |37.88   |41.0              |880.0      |129.0         |322.0     |126.0     |8.3252       |452600.0          |NEAR BAY       |
|-122.22  |37.86   |21.0              |7099.0     |1106.0        |2401.0    |1138.0    |8.3014       |358500.0          |NEAR BAY       |
|-122.24  |37.85   |52.0              |1467.0     |190.0         |496.0     |177.0     |7.2574       |352100.0          |NEAR BAY       |
|-122.25  |37.85   |52.0              |1274.0     |235.0         |558.0     |219.0     |5.6431       |341300.0          |NEAR BAY       |
|-122.25  |37.85   |52.0          

Print the number of records with population more than 10000.

In [5]:
housing.where("population > 10000").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -121.92|   37.53|               7.0|    28258.0|        3864.0|   12203.0|    3701.0|       8.4045|          451100.0|      <1H OCEAN|
|  -117.78|   34.03|               8.0|    32054.0|        5290.0|   15507.0|    5050.0|       6.0191|          253900.0|      <1H OCEAN|
|  -117.87|   34.04|               7.0|    27700.0|        4179.0|   15037.0|    4072.0|       6.6288|          339700.0|      <1H OCEAN|
|  -117.88|   33.96|              16.0|    19059.0|        3079.0|   10988.0|    3061.0|       5.5469|          265200.0|      <1H OCEAN|
|  -118.78|   34.16|              

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [6]:
housing.describe("housing_median_age", "total_rooms", "median_house_value", "population").show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [7]:
import org.apache.spark.sql.functions._

housing.agg(max("housing_median_age"),min("total_rooms"),avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



import org.apache.spark.sql.functions._


## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [8]:
housing.groupBy("ocean_proximity").count().orderBy(desc("count")).show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [9]:
housing.groupBy("ocean_proximity").agg(avg("median_house_value").as("avg_value")).show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [10]:
housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, AVG(median_house_value) AS avg_value FROM df GROUP BY ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [11]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("housing_median_age", "total_rooms", "median_house_value","population"))
    .setOutputCol("features_temp")

val housingAttrs = va.transform(housing)
housingAttrs.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|       features_temp|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|[41.0,880.0,45260...|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|[21.0,7099.0,3585...|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|[52.0,1467.0,3521...|
|  -122.25|   37.85|              52.0|     12

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_14d7869b9726
housingAttrs: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]


In [12]:
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(housingAttrs,"features_temp").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293   
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772    
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235  
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0                   


import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0                   -0.36126220122231784  0.10562341249318154   -0.2962442397735293
-0.36126220122231784  1.0                   0.13415311380654338   0.8571259728659772
0.10562341249318154   0.13415311380654338   1.0                   -0.02464967888891235
-0.2962442397735293   0.8571259728659772    -0.02464967888891235  1.0


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [13]:
val housingCol1 = housing.withColumn("rooms_per_household", $"total_rooms" / $"households")
val housingCol2 = housingCol1.withColumn("bedrooms_per_room", $"total_bedrooms" / $"total_rooms")
val housingExtra = housingCol2.withColumn("population_per_household",$"population" / $"households")

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



housingCol1: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 9 more fields]
housingCol2: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 10 more fields]
housingExtra: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [14]:
val renamedHousing = housingExtra.withColumnRenamed("median_house_value","label")

renamedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [15]:
// label columns
val colLabel = "label"

// categorical columns
val colCat = "ocean_proximity"

// numerical columns
val colNum = renamedHousing.columns.filter(_ != colLabel).filter(_ != colCat)

colLabel: String = label
colCat: String = ocean_proximity
colNum: Array[String] = Array(longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, rooms_per_household, bedrooms_per_room, population_per_household)


## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [16]:
for (c <- colNum) {
    print(c + " ")
    println(renamedHousing.select(c).where(c+" IS NULL").count())
}

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
rooms_per_household 0
bedrooms_per_room 207
population_per_household 0


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [17]:
import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer().setStrategy("median").setInputCols(Array("total_bedrooms","bedrooms_per_room")).setOutputCols(Array("total_bedrooms","bedrooms_per_room"))                       
val imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)
imputedHousing.select("*").show(5)


+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129

import org.apache.spark.ml.feature.Imputer
imputer: org.apache.spark.ml.feature.Imputer = imputer_944b37b07490
imputedHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 11 more fields]


### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [18]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(Array("longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","rooms_per_household","bedrooms_per_room","population_per_household"))
    .setOutputCol("features_temp")
val featuredHousing = va.transform(imputedHousing)

val scaler = new StandardScaler().setInputCol("features_temp").setOutputCol("scaled_features_temp").setWithStd(true)
val scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|       features_temp|scaled_features_temp|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_55f0a5e309f4
featuredHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_b5147144cca4
scaledHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]


## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [19]:
renamedHousing.select("ocean_proximity").agg(countDistinct("ocean_proximity").as("count distinct ocean_proximity")).show

+------------------------------+
|count distinct ocean_proximity|
+------------------------------+
|                             5|
+------------------------------+



### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [20]:
import org.apache.spark.ml.feature.StringIndexer

val indexer = new StringIndexer().setInputCol("ocean_proximity").setOutputCol("ocean_proximity_numerical")
val idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_numerical|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                      3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR B

import org.apache.spark.ml.feature.StringIndexer
indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_19b6434fdd83
idxHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 12 more fields]


Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [21]:
indexer.fit(renamedHousing).labels

res17: Array[String] = Array(<1H OCEAN, INLAND, NEAR OCEAN, NEAR BAY, ISLAND)


### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [22]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("ocean_proximity_numerical")).setOutputCols(Array("ocean_proximity_onehot"))
val ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------------+----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|ocean_proximity_numerical|ocean_proximity_onehot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-------------------------+----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                      3.0|         (4,[3],[1.0])|
|  -122.22|   37.86|            

import org.apache.spark.ml.feature.OneHotEncoderEstimator
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_b4779ca11c2b
ohHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 13 more fields]


---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [23]:
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(imputer,va,scaler))
val catPipeline = new Pipeline().setStages(Array(indexer,encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-------------------------+----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|       features_temp|scaled_features_temp|ocean_proximity_numerical|ocean_proximity_onehot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-------------------------+----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
numPipeline: org.apache.spark.ml.Pipeline = pipeline_f0d533b8e6b8
catPipeline: org.apache.spark.ml.Pipeline = pipeline_2d649ae1a7cd
pipeline: org.apache.spark.ml.Pipeline = pipeline_75c5aab10d3e
newHousing: org.apache.spark.sql.DataFrame = [longitude: double, latitude: double ... 15 more fields]


Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [24]:
val va2 = new VectorAssembler()
              .setInputCols(Array("scaled_features_temp","ocean_proximity_onehot"))
              .setOutputCol("features")
val dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5,false)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|features                                                                                                                                                                                                                            |label   |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|[-61.00726959606955,17.734477624640412,3.2577023016083064,0.40337085073160667,0.30758821710917267,0.2843362208866199,0.3295584480852433,4.382095394195227,2.8228125480951665,2.5405867237343416,0.24605655309533123,0.0,0.0,0.0,1.0]|452600.0|
|[-61.002278409814444,17.725114120086744

va2: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_8502a2faf93d
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: double]


---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [25]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8,0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: double]


## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [26]:
import org.apache.spark.ml.regression.LinearRegression

// train the model
val lr = new LinearRegression()
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.summary

println(s"Coefficients: ${lrModel.coefficients}, Intercept: ${lrModel.intercept}")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")

Coefficients: [-56448.81022555011,-56550.35366710437,13366.938079371965,8285.862275205669,76.82597894565546,-46120.01187223349,43274.240891836096,77451.09662465853,6887.001239647308,16798.338592429714,3146.939480202045,-123758.68669962902,-158643.2734272458,-121782.16586696701,-130402.28590906346], Intercept: -2358520.37258488
RMSE: 67587.35999776557


import org.apache.spark.ml.regression.LinearRegression
lr: org.apache.spark.ml.regression.LinearRegression = linReg_b09ca5b62989
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_b09ca5b62989
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@547e82d8


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [27]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 177861.2729229792| 79000.0|[-62.025471592109...|
|206966.67809458124|106700.0|[-62.005506847089...|
| 120357.3010686473| 68400.0|[-61.995524474579...|
|191484.31032353826| 90100.0|[-61.985542102068...|
|169408.39543577842| 69000.0|[-61.985542102068...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 69375.7549109374


import org.apache.spark.ml.evaluation.RegressionEvaluator
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_d811794220b4
rmse: Double = 69375.7549109374


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [28]:
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|167306.13913043478| 79000.0|[-62.025471592109...|
|167306.13913043478|106700.0|[-62.005506847089...|
|167306.13913043478| 68400.0|[-61.995524474579...|
| 194426.6505551927| 90100.0|[-61.985542102068...|
|148875.44642857142| 69000.0|[-61.985542102068...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 68980.6368931534


import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
dt: org.apache.spark.ml.regression.DecisionTreeRegressor = dtr_6c2beaaba4a8
dtModel: org.apache.spark.ml.regression.DecisionTreeRegressionModel = DecisionTreeRegressionModel (uid=dtr_6c2beaaba4a8) of depth 5 with 63 nodes
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_7998a7c2baf5
rmse: Double = 68980.6368931534


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [29]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|151715.39036059403| 79000.0|[-62.025471592109...|
| 170545.3372775281|106700.0|[-62.005506847089...|
| 148268.6202436621| 68400.0|[-61.995524474579...|
|196553.40804232968| 90100.0|[-61.985542102068...|
|159210.13396455412| 69000.0|[-61.985542102068...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 67089.58739307892


import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_7ba1237d723d
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel (uid=rfr_7ba1237d723d) with 20 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_817103497f29
rmse: Double = 67089.58739307892


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [30]:
import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val gb = new GBTRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val gbModel = gb.fit(trainSet)

// make predictions on the test data
val predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 94445.48912281399| 79000.0|[-62.025471592109...|
|127579.29928621747|106700.0|[-62.005506847089...|
|129471.46392211645| 68400.0|[-61.995524474579...|
| 89184.91128900625| 90100.0|[-61.985542102068...|
| 85628.86441601116| 69000.0|[-61.985542102068...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 56283.18784790383


import org.apache.spark.ml.regression.GBTRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
gb: org.apache.spark.ml.regression.GBTRegressor = gbtr_51a964473a14
gbModel: org.apache.spark.ml.regression.GBTRegressionModel = GBTRegressionModel (uid=gbtr_51a964473a14) with 20 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: double ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = regEval_8b2367271a22
rmse: Double = 56283.18784790383


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [31]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder().addGrid(rf.numTrees,Array(1,5,10)).addGrid(rf.maxDepth,Array(5,10,15)).build()

val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val cv = new CrossValidator().setEstimator(rf).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(3)
val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|   92560.648989899| 79000.0|[-62.025471592109...|
|186203.76666666666|106700.0|[-62.005506847089...|
|113573.27450980393| 68400.0|[-61.995524474579...|
| 171889.5825471481| 90100.0|[-61.985542102068...|
| 91259.47463768117| 69000.0|[-61.985542102068...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 53856.9134137876


import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfr_7ba1237d723d-maxDepth: 5,
	rfr_7ba1237d723d-numTrees: 1
}, {
	rfr_7ba1237d723d-maxDepth: 5,
	rfr_7ba1237d723d-numTrees: 5
}, {
	rfr_7ba1237d723d-maxDepth: 5,
	rfr_7ba1237d723d-numTrees: 10
}, {
	rfr_7ba1237d723d-maxDepth: 10,
	rfr_7ba1237d723d-numTrees: 1
}, {
	rfr_7ba1237d723d-maxDepth: 10,
	rfr_7ba1237d723d-numTrees: 5
}, {
	rfr_7ba1237d723d-maxDepth: 10,
	rfr_7ba1237d723d-numTrees: 10
}, {
	rfr_7ba1237d723d-maxDepth: 15,
	rfr_7ba1237d723d-numTrees: 1
}, {
	rfr_7ba1237d723d-maxDepth: 15,
	rfr_7ba1237d723d-numTrees: 5
}, {
	rfr_7ba1237...

---
# 7. Custom transformer
At the end of part two, we added extra columns to the `housing` dataset. Here, we are going to implement a Transformer to do the same task. The Transformer should take the name of two input columns `inputCol1` and `inputCol2`, as well as the name of ouput column `outputCol`. It, then, computes `inputCol1` divided by `inputCol2`, and adds its result as a new column to the dataset. The details of the implemeting a custom Tranfomer is explained [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types). Please read it before before starting to implement it.

First, define the given parameters of the Transformer and implement a method to validate their schemas (`StructType`).

In [32]:
import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}

trait MyParams extends Params {
    final val inputCol1 = new Param[String](this, "inputCol1", "The first input column")
    final val inputCol2 = new Param[String](this, "inputCol2", "The second input column")
    final val outputCol = new Param[String](this, "outputCol", "The output column")
    
  protected def validateAndTransformSchema(schema: StructType): StructType = {
    val indexInputCol1 = schema.fieldIndex("inputCol1") 
    val fieldInputCol1 = schema.fields(indexInputCol1)
      
    if (fieldInputCol1.dataType != DoubleType) {
      throw new Exception(s"Input type ${fieldInputCol1.dataType} did not match input type DoubleType")
    }
    
    val indexInputCol2 = schema.fieldIndex("inputCol2")    
    val fieldInputCol2 = schema.fields(indexInputCol2)

    if (fieldInputCol2.dataType != DoubleType) {
      throw new Exception(s"Input type ${fieldInputCol2.dataType} did not match input type DoubleType")
    }
      
    schema.add(StructField("division", DoubleType, false))
  }
}


import org.apache.spark.sql.types.{StructField, StructType, DoubleType}
import org.apache.spark.ml.param.{ParamMap, Param, Params}
defined trait MyParams


Then, extend the class `Transformer`, and implement its setter functions for the input and output columns, and call then `setInputCol1`, `setInputCol2`, and `setOutputCol`. Morever, you need to override the methods `copy`, `transformSchema`, and the `transform`. The details of what you need to cover in these methods is given [here](https://www.oreilly.com/learning/extend-spark-ml-for-your-own-modeltransformer-types).

In [33]:
import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}

class MyTransformer(override val uid: String) extends Transformer with MyParams {
    def this() = this(Identifiable.randomUID("superbeautifultransformer"))
    
    def setInputCol1(value: String): this.type = set(inputCol1, value)
    
    def setInputCol2(value: String): this.type = set(inputCol2, value)
    
    def setOutputCol(value: String): this.type = set(outputCol, value)

    override def copy(extra: ParamMap): MyTransformer = {
        defaultCopy(extra)
    }
    override def transformSchema(schema: StructType): 
        StructType = {
            validateAndTransformSchema(schema)
        }
    
    override def transform(dataset: Dataset[_]): DataFrame = {
        val col1 = dataset.col($(inputCol1))
        val col2 = dataset.col($(inputCol2))
        dataset.withColumn($(outputCol), col1 / col2)
    }
}

import org.apache.spark.ml.util.Identifiable
import org.apache.spark.ml.Transformer
import org.apache.spark.ml.param.{ParamMap, Param, Params}
import org.apache.spark.sql.{DataFrame, Dataset}
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.functions.{col, udf}
defined class MyTransformer


Now, an instance of `MyTransformer`, and set the input columns `total_rooms` and `households`, and the output column `rooms_per_household` and run it over the `housing` dataset.

In [34]:
val myTransformer = new MyTransformer().setInputCol1("total_rooms").setInputCol2("households").setOutputCol("rooms_per_household")

val myDataset = myTransformer.transform(housing).select("rooms_per_household").show(5)

+-------------------+
|rooms_per_household|
+-------------------+
|  6.984126984126984|
|  6.238137082601054|
|  8.288135593220339|
| 5.8173515981735155|
|  6.281853281853282|
+-------------------+
only showing top 5 rows



myTransformer: MyTransformer = superbeautifultransformer_0179190ac832
myDataset: Unit = ()


---
# 8. Custom estimator (predictor)
Now, it's time to implement your own linear regression with gradient descent algorithm as a brand new Estimator. The whole code of the Estimator is given to you, and you do not need to implement anything. It is just a sample that shows how to build a custom Estimator.

The gradient descent update for linear regression is:
$$
w_{i+1} = w_{i} - \alpha_{i} \sum\limits_{j=1}^n (w_i^\top x_j - y_j)x_j
$$

where $i$ is the iteration number of the gradient descent algorithm, and $j$ identifies the observation. Here, $w$ represents an array of weights that is the same size as the array of features and provides a weight for each of the features when finally computing the label prediction in the form:

$$
prediction = w^\top \cdot\ x
$$

where $w$ is the final array of weights computed by the gradient descent, $x$ is the array of features of the observation point and $prediction$ is the label we predict should be associated to this observation.

The given `Helper` class implements the helper methods:
* `dot`: implements the dot product of two vectors and the dot product of a vector and a scalar
* `sum`: implements addition of two vectors
* `fill`: creates a vector of predefined size and initialize it with the predefined value

What you need to do is to implement the methods of the Linear Regresstion class `LR`, which are
* `rmsd`: computes the Root Mean Square Error of a given RDD of tuples of (label, prediction) using the formula:
$$
rmse = \sqrt{\frac{\sum\limits_{i=1}^n (label - prediction)^2}{n}}
$$
* `gradientSummand`: computes the following formula:
$$
gs_{ij} = (w_i^\top x_j - y_j)x_j
$$
* `gradient`: computes the following formula:
$$
gradient = \sum\limits_{j=1}^n gs_{ij}
$$

In [35]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

case class Instance(label: Double, features: Vector)

object Helper extends Serializable {
  def dot(v1: Vector, v2: Vector): Double = {
    val m = Matrices.dense(1, v1.size, v1.toArray)
    m.multiply(v2).values(0)
  }

  def dot(v: Vector, s: Double): Vector = {
    val baseArray = v.toArray.map(vi => vi * s)
    Vectors.dense(baseArray)
  }

  def sumVectors(v1: Vector, v2: Vector): Vector = {
    val baseArray = ((v1.toArray) zip (v2.toArray)).map { case (val1, val2) => val1 + val2 }
    Vectors.dense(baseArray)
  }

  def fillVector(size: Int, fillVal: Double): Vector = Vectors.dense(Array.fill[Double](size)(fillVal));
}

import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
defined class Instance
defined object Helper


In [36]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

class LR() extends Serializable {
  def calcRMSE(labelsAndPreds: RDD[(Double, Double)]): Double = {
    val regressionMetrics = new RegressionMetrics(labelsAndPreds)
    regressionMetrics.rootMeanSquaredError
  }
  
  def gradientSummand(weights: Vector, lp: Instance): Vector = {
    val mult = (Helper.dot(weights, lp.features) - lp.label)
    val seq = (0 to lp.features.size - 1).map(i => lp.features(i) * mult)
    return Vectors.dense(seq.toArray)
  }
  
  def linregGradientDescent(trainData: RDD[Instance], numIters: Int): (Vector, Array[Double]) = {
    val n = trainData.count()
    val d = trainData.take(1)(0).features.size
    var w = Helper.fillVector(d, 0)
    val alpha = 1.0
    val errorTrain = Array.fill[Double](numIters)(0.0)

    for (i <- 0 until numIters) {
      val labelsAndPredsTrain = trainData.map(lp => (lp.label, Helper.dot(w, lp.features)))
      errorTrain(i) = calcRMSE(labelsAndPredsTrain)

      val gradient = trainData.map(lp => gradientSummand(w, lp)).reduce((v1, v2) => Helper.sumVectors(v1, v2))
      val alpha_i = alpha / (n * scala.math.sqrt(i + 1))
      val wAux = Helper.dot(gradient, (-1) * alpha_i)
      w = Helper.sumVectors(w, wAux)
    }
    (w, errorTrain)
  }
}

import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
defined class LR


In [37]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearModel[FeaturesType, Model <: MyLinearModel[FeaturesType, Model]]
  extends PredictionModel[FeaturesType, Model] {
}

class MyLinearModelImpl(override val uid: String, val weights: Vector, val trainingError: Array[Double])
    extends MyLinearModel[Vector, MyLinearModelImpl] {

  override def copy(extra: ParamMap): MyLinearModelImpl = defaultCopy(extra)

  def predict(features: Vector): Double = {
    println("Predicting")
    val prediction = Helper.dot(weights, features)
    prediction
  }
}

import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
defined class MyLinearModel
defined class MyLinearModelImpl


In [38]:
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._

import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}

abstract class MyLinearRegression[
    FeaturesType,
    Learner <: MyLinearRegression[FeaturesType, Learner, Model],
    Model <: MyLinearModel[FeaturesType, Model]]
  extends Predictor[FeaturesType, Learner, Model] {
}

class MyLinearRegressionImpl(override val uid: String)
    extends MyLinearRegression[Vector, MyLinearRegressionImpl, MyLinearModelImpl] {
  def this() = this(Identifiable.randomUID("linReg"))

  override def copy(extra: ParamMap): MyLinearRegressionImpl = defaultCopy(extra)
  
  def train(dataset: Dataset[_]): MyLinearModelImpl = {
    println("Training")

    val numIters = 10

    val instances: RDD[Instance] = dataset.select(
      col($(labelCol)), col($(featuresCol))).rdd.map {
        case Row(label: Double, features: Vector) =>
          Instance(label, features)
      }

    val (weights, trainingError) = new LR().linregGradientDescent(instances, numIters)

    new MyLinearModelImpl(uid, weights, trainingError)
  }
}

import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.PredictorParams
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.util._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Row
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.linalg.Matrices
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.ml.{PredictionModel, Predictor}
defined class MyLinearRegression
defined class MyLinearRegressionImpl


In [39]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val lr = new MyLinearRegressionImpl().setLabelCol("label").setFeaturesCol("features")
val model = lr.fit(trainSet)
val predictions = model.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// Select (prediction, true label) and compute test error.
val evaluator = new RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
val rmse = evaluator.evaluate(predictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")

Training


org.apache.spark.SparkDriverExecutionException:  Execution error

##### ---
# 9. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

## 1. Load the data.

We read the dataset and rename the column DEFAULT as label.

In [1]:
val cards = spark.read.format("csv").option("header","true").option("inferSchema","true").load("data/ccdefault.csv")

val renamedCards = cards.withColumnRenamed("DEFAULT","label")

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-QRC7UDIG-di-Riccardo:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1574416260023)
SparkSession available as 'spark'


cards: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]
renamedCards: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]


## 2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).


First of all, we want to understand what the dataset contains. In order to do so we print the Schema of it and the first 5 rows.

In [2]:
renamedCards.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- label: integer (nullable = true)

In [3]:
renamedCards.select("*").show(5,false)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|ID |LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|1  |20000    |2  |2        |1       |24 |2    |2    |-1   |-1   |-2   |-2   |3913     |3102     |689      |0        |0        |0        |0       |689     |0       |0       |0       |0       |1    |
|2  |120000   |2  |2        |2       |26 |-1   |2    |0    |0    |0    |2    |2682     |1725     |2682     |3272     |3455     |3261     |0       |1000    |1000    |1000    |0       |2000    |1    |
|3  |

We want to check the distribution of the target variable. In order to do so we count the number of samples with default value 1 and 0

In [4]:
renamedCards.groupBy("label").count().withColumn("percent",col("count")/30000).show()

+-----+-----+-------+
|label|count|percent|
+-----+-----+-------+
|    1| 6636| 0.2212|
|    0|23364| 0.7788|
+-----+-----+-------+



As we can see, the number of non-default samples are almost 4 times larger than the default. Therefore, we cannot refer only to the accuracy metric to evaluate the trained model but we have to use other metrics such as the area under the ROC curve.

Below we further perform the exploratory analysis by getting the count, mean, standard deviation, min and max value of the attributes LIMIT_BAL.

In [5]:
renamedCards.describe("LIMIT_BAL").show()

+-------+------------------+
|summary|         LIMIT_BAL|
+-------+------------------+
|  count|             30000|
|   mean|167484.32266666667|
| stddev|129747.66156720246|
|    min|             10000|
|    max|           1000000|
+-------+------------------+



If given credit would be normally distributed the mean would be 505,000. Here we can observe that the mean is significantly lower meaning that the limits are usually lower than the mid value.

In [6]:
renamedCards.groupBy("SEX").count().withColumn("percent",col("count")/30000).show()

+---+-----+-------------------+
|SEX|count|            percent|
+---+-----+-------------------+
|  1|11888|0.39626666666666666|
|  2|18112| 0.6037333333333333|
+---+-----+-------------------+



Here we can observe that there are more females than males.

In [7]:
renamedCards.groupBy("EDUCATION").count().withColumn("percent",col("count")/30000).orderBy("EDUCATION").show(false)

+---------+-----+---------------------+
|EDUCATION|count|percent              |
+---------+-----+---------------------+
|0        |14   |4.6666666666666666E-4|
|1        |10585|0.35283333333333333  |
|2        |14030|0.4676666666666667   |
|3        |4917 |0.1639               |
|4        |123  |0.0041               |
|5        |280  |0.009333333333333334 |
|6        |51   |0.0017               |
+---------+-----+---------------------+



The education values are between 0 and 6, and not from 1 to 4, as described in the txt file.

In [8]:
for (c <- renamedCards.columns) {
    print(c + " ")
    println(renamedCards.select(c).where(c+" IS NULL").count())
}

ID 0
LIMIT_BAL 0
SEX 0
EDUCATION 0
MARRIAGE 0
AGE 0
PAY_0 0
PAY_2 0
PAY_3 0
PAY_4 0
PAY_5 0
PAY_6 0
BILL_AMT1 0
BILL_AMT2 0
BILL_AMT3 0
BILL_AMT4 0
BILL_AMT5 0
BILL_AMT6 0
PAY_AMT1 0
PAY_AMT2 0
PAY_AMT3 0
PAY_AMT4 0
PAY_AMT5 0
PAY_AMT6 0
label 0


Finally, there are no missing values in the dataset, so we do not need an inputer as a preprocessing step.

## 3. Train a model to predict the target variable (risk of default).

### 3.1 Preprocessing

First of all, we want to eliminate the ID column, as it is not relevant for the model we will train.

In [9]:
val cardsNoID = renamedCards.drop("ID")
cardsNoID.printSchema()

root
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- label: integer (nullable = true)



cardsNoID: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]


For categorical data, we want to use One-hot encoding, the reason is that the models we are going to use will not work correctly with categorical non-binary features. For example, if the model internally calculate an average the result might be wrong.

In [20]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.feature.StringIndexer

val indexer0 = new StringIndexer()
  .setInputCol("PAY_0")
  .setOutputCol("PAY_0_INDEX")

val indexed0 = indexer0.fit(cardsNoID).transform(cardsNoID)

val indexer2 = new StringIndexer()
  .setInputCol("PAY_2")
  .setOutputCol("PAY_2_INDEX")

val indexed2 = indexer2.fit(indexed0).transform(indexed0)
val indexer3 = new StringIndexer()
  .setInputCol("PAY_3")
  .setOutputCol("PAY_3_INDEX")

val indexed3 = indexer3.fit(indexed2).transform(indexed2)
val indexer4 = new StringIndexer()
  .setInputCol("PAY_4")
  .setOutputCol("PAY_4_INDEX")

val indexed4 = indexer4.fit(indexed3).transform(indexed3)
val indexer5 = new StringIndexer()
  .setInputCol("PAY_5")
  .setOutputCol("PAY_5_INDEX")

val indexed5 = indexer5.fit(indexed4).transform(indexed4)
val indexer6 = new StringIndexer()
  .setInputCol("PAY_6")
  .setOutputCol("PAY_6_INDEX")

val indexed6 = indexer6.fit(indexed5).transform(indexed5)

val encoder = new OneHotEncoderEstimator().setInputCols(Array("SEX","EDUCATION","MARRIAGE","PAY_0_INDEX","PAY_2_INDEX","PAY_3_INDEX","PAY_4_INDEX","PAY_5_INDEX","PAY_6_INDEX"))
     .setOutputCols(Array("SEX_ONEHOT","EDUCATION_ONEHOT","MARRIAGE_ONEHOT","PAY_0_ONEHOT","PAY_2_ONEHOT","PAY_3_ONEHOT","PAY_4_ONEHOT","PAY_5_ONEHOT","PAY_6_ONEHOT"))
val ohCardsNoID = encoder.fit(indexed6).transform(indexed6)
ohCardsNoID.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-----------+-----------+-----------+-----------+-----------+-----------+--------------+-------------+--------------+--------------+-------------+--------------+---------------+-------------+----------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|PAY_0_INDEX|PAY_2_INDEX|PAY_3_INDEX|PAY_4_INDEX|PAY_5_INDEX|PAY_6_INDEX|  PAY_4_ONEHOT| PAY_5_ONEHOT|  PAY_0_ONEHOT|  PAY_3_ONEHOT| PAY_6_ONEHOT|  PAY_2_ONEHOT|MARRIAGE_ONEHOT|   SEX_ONEHOT|EDUCATION_ONEHOT|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--

import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.feature.StringIndexer
indexer0: org.apache.spark.ml.feature.StringIndexer = strIdx_908205006369
indexed0: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 23 more fields]
indexer2: org.apache.spark.ml.feature.StringIndexer = strIdx_d2357087e4c3
indexed2: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 24 more fields]
indexer3: org.apache.spark.ml.feature.StringIndexer = strIdx_90cfaa241a48
indexed3: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 25 more fields]
indexer4: org.apache.spark.ml.feature.StringIndexer = strIdx_30068f4d21e2
indexed4: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 26 more fields]
indexer5: org.apache.spark.m...

As the dataset contains redundant columns we will drop the columns PAY_X and PAY_X_INDEX

In [21]:
val ohcardsNoPAY = ohCardsNoID.drop("PAY_0_INDEX","PAY_2_INDEX","PAY_3_INDEX","PAY_4_INDEX","PAY_5_INDEX","PAY_6_INDEX","PAY_0","PAY_2","PAY_3","PAY_4","PAY_5","PAY_6")

ohcardsNoPAY.show()

+---------+---+---------+--------+---+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------+-------------+--------------+--------------+-------------+--------------+---------------+-------------+----------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|  PAY_4_ONEHOT| PAY_5_ONEHOT|  PAY_0_ONEHOT|  PAY_3_ONEHOT| PAY_6_ONEHOT|  PAY_2_ONEHOT|MARRIAGE_ONEHOT|   SEX_ONEHOT|EDUCATION_ONEHOT|
+---------+---+---------+--------+---+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------+-------------+--------------+--------------+-------------+--------------+---------------+-------------+----------------+
|    20000|  2|        2|       1| 24|     3913|     3102|      689|        0|        0|        0|       0|     689|  

ohcardsNoPAY: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 25 more fields]


The next operation we want to define is scaling. It is proven that scaled features help models to train and generalise better.

In [22]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

// take all the columns but the ones defined in the array
val colNum = ohcardsNoPAY.columns.filter(!Array("SEX","MARRIAGE","EDUCATION","SEX_ONEHOT", "EDUCATION_ONEHOT", "MARRIAGE_ONEHOT","PAY_0_ONEHOT","PAY_2_ONEHOT","PAY_3_ONEHOT","PAY_4_ONEHOT","PAY_5_ONEHOT","PAY_6_ONEHOT","label").contains(_))

val va = new VectorAssembler().setInputCols(colNum)
    .setOutputCol("features_temp")
val featuredCards = va.transform(ohcardsNoPAY)

val scaler = new StandardScaler().setInputCol("features_temp").setOutputCol("scaled_features_temp").setWithStd(true)
val scaledCards = scaler.fit(featuredCards).transform(featuredCards)

scaledCards.select("scaled_features_temp").show(10, false)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled_features_temp                                                                                                                                                                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(14,[0,1,2,3,4,9],[0.15414535998894324,2.603628744963987,0.053139869207970765,0.0435834725779124,0.009935199510232218,0.029903384202815683])                   

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
colNum: Array[String] = Array(LIMIT_BAL, AGE, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_cdf630f0f4d9
featuredCards: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 26 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_bca1cefd3ae1
scaledCards: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 27 more fields]


As a final preprocessing step, we want the dataset to have only 2 columns: features and label.

In [23]:
val va2 = new VectorAssembler()
              .setInputCols(Array("SEX_ONEHOT","EDUCATION_ONEHOT","MARRIAGE_ONEHOT","PAY_0_ONEHOT","PAY_2_ONEHOT","PAY_3_ONEHOT","PAY_4_ONEHOT","PAY_5_ONEHOT","PAY_6_ONEHOT","scaled_features_temp"))
              .setOutputCol("features")

val dataset = va2.transform(scaledCards).select("features", "label")

dataset.show(5,false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                     |label|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

va2: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_c9f4fa3f0709
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: int]


We transformed both categorical and numverical features using the OneHotEncoderEstimator, the VectorArrassember and the StandardScaler.
Rather then using all these steps manually we could just define the transformers and add them to a pipeline and execute it.

In [24]:
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val catPipeline = new Pipeline().setStages(Array(encoder))
val numPipeline = new Pipeline().setStages(Array(va,scaler))
val featuresPipeline = new Pipeline().setStages(Array(va2))
val pipeline = new Pipeline().setStages(Array(catPipeline, numPipeline,featuresPipeline))
val newCards = pipeline.fit(indexed6).transform(indexed6).select("features", "label")

newCards.show(5,false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                     |label|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
catPipeline: org.apache.spark.ml.Pipeline = pipeline_3a59070569f2
numPipeline: org.apache.spark.ml.Pipeline = pipeline_4a8d770f29ab
featuresPipeline: org.apache.spark.ml.Pipeline = pipeline_7b22a328c747
pipeline: org.apache.spark.ml.Pipeline = pipeline_6ba43e800430
newCards: org.apache.spark.sql.DataFrame = [features: vector, label: int]


As it can be seen, the result is exactly the same.

### 3.2 Model fit

We are now ready to fit some Models. First of all, we split the dataset into 2 parts, train and test sets. We use the 80% of the dataset for training and the remaining for testing.

In [25]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8,0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]


#### 3.2.1 Logistic Regression

In [118]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

// train the model
val lr = new LogisticRegression()
val lrModel = lr.fit(trainSet)

// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error.
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")
val areaUnderROC = evaluator.evaluate(predictions)
println(s"areaUnderROC on test data = $areaUnderROC")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(83,[1,2,10,11,21...|
|       0.0|    0|(83,[1,3,9,11,21,...|
|       0.0|    1|(83,[1,3,9,11,21,...|
|       0.0|    0|(83,[1,3,9,11,21,...|
|       0.0|    0|(83,[1,3,9,11,21,...|
+----------+-----+--------------------+
only showing top 5 rows

areaUnderROC on test data = 0.6464000150038448


import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_86d44421c3d5
lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_86d44421c3d5, numClasses = 2, numFeatures = 83
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_530e2595a4dc
areaUnderROC: Double = 0.6464000150038448


#### 3.2.2 Decision Tree

In [122]:
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.mllib.util.MLUtils

val dt = new DecisionTreeClassifier().setLabelCol("label").setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)
val areaUnderROC = evaluator.evaluate(predictions)
println(s"areaUnderROC on test data = $areaUnderROC")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(83,[1,2,10,11,21...|
|       0.0|    0|(83,[1,3,9,11,21,...|
|       0.0|    1|(83,[1,3,9,11,21,...|
|       0.0|    0|(83,[1,3,9,11,21,...|
|       0.0|    0|(83,[1,3,9,11,21,...|
+----------+-----+--------------------+
only showing top 5 rows

areaUnderROC on test data = 0.6383525201409784


import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.tuning.CrossValidator
import org.apache.spark.mllib.tree.model.DecisionTreeModel
import org.apache.spark.mllib.util.MLUtils
dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_73f0bc4db4b1
dtModel: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_73f0bc4db4b1) of depth 5 with 49 nodes
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
areaUnderROC: Double = 0.6383525201409784


#### 3.2.3 Param Grid for the Decision Tree

In [124]:
val paramGrid = new ParamGridBuilder().addGrid(dt.maxBins,Array(8, 16, 32, 64, 128)).addGrid(dt.maxDepth,Array(30)).build()

// select (prediction, true label) and compute test error
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")
val cv = new CrossValidator().setEstimator(dt).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(3)
val cvModel = cv.fit(trainSet)


val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val areaUnderROC = evaluator.evaluate(predictions)
println(s"areaUnderROC on test data = $areaUnderROC")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(83,[1,2,10,11,21...|
|       0.0|    0|(83,[1,3,9,11,21,...|
|       0.0|    1|(83,[1,3,9,11,21,...|
|       0.0|    0|(83,[1,3,9,11,21,...|
|       1.0|    0|(83,[1,3,9,11,21,...|
+----------+-----+--------------------+
only showing top 5 rows

areaUnderROC on test data = 0.6212507019026189


paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	dtc_73f0bc4db4b1-maxBins: 8,
	dtc_73f0bc4db4b1-maxDepth: 30
}, {
	dtc_73f0bc4db4b1-maxBins: 16,
	dtc_73f0bc4db4b1-maxDepth: 30
}, {
	dtc_73f0bc4db4b1-maxBins: 32,
	dtc_73f0bc4db4b1-maxDepth: 30
}, {
	dtc_73f0bc4db4b1-maxBins: 64,
	dtc_73f0bc4db4b1-maxDepth: 30
}, {
	dtc_73f0bc4db4b1-maxBins: 128,
	dtc_73f0bc4db4b1-maxDepth: 30
})
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_770963dedb75
cv: org.apache.spark.ml.tuning.CrossValidator = cv_f4671d5b6f35
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_f4671d5b6f35
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
areaUnderROC: Double = 0.6212507019026189


#### 3.2.4 Random Forest

In [104]:
import org.apache.spark.ml.regression.RandomForestRegressor

val rf = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")
val areaUnderROC = evaluator.evaluate(predictions)
println(s"areaUnderROC on test data = $areaUnderROC")

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|0.09361841055670166|    0|(83,[1,2,10,11,21...|
|0.10208701021894953|    0|(83,[1,3,9,11,21,...|
| 0.1715258223392432|    1|(83,[1,3,9,11,21,...|
|0.14282535838057392|    0|(83,[1,3,9,11,21,...|
| 0.1644713353749631|    0|(83,[1,3,9,11,21,...|
+-------------------+-----+--------------------+
only showing top 5 rows

areaUnderROC on test data = 0.7731179832780879


import org.apache.spark.ml.regression.RandomForestRegressor
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_d9118a3f983e
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel (uid=rfr_d9118a3f983e) with 20 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_977727afb905
areaUnderROC: Double = 0.7731179832780879


#### 3.2.5 Param Grid for the Random Forest

In [106]:
import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator

val paramGrid = new ParamGridBuilder().addGrid(rf.numTrees,Array(1,5,10)).addGrid(rf.maxDepth,Array(5,10,15)).build()

val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")
val cv = new CrossValidator().setEstimator(rf).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(3)
val cvModel = cv.fit(trainSet)

val predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val areaUnderROC = evaluator.evaluate(predictions)
println(s"areaUnderROC on test data = $areaUnderROC")

+--------------------+-----+--------------------+
|          prediction|label|            features|
+--------------------+-----+--------------------+
|0.058045247581142025|    0|(83,[1,2,10,11,21...|
| 0.09301696910423396|    0|(83,[1,3,9,11,21,...|
| 0.19592123628425184|    1|(83,[1,3,9,11,21,...|
| 0.12944545889114104|    0|(83,[1,3,9,11,21,...|
|  0.1457156554910614|    0|(83,[1,3,9,11,21,...|
+--------------------+-----+--------------------+
only showing top 5 rows

areaUnderROC on test data = 0.7757382059922101


import org.apache.spark.ml.tuning.ParamGridBuilder
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.CrossValidator
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfr_d9118a3f983e-maxDepth: 5,
	rfr_d9118a3f983e-numTrees: 1
}, {
	rfr_d9118a3f983e-maxDepth: 5,
	rfr_d9118a3f983e-numTrees: 5
}, {
	rfr_d9118a3f983e-maxDepth: 5,
	rfr_d9118a3f983e-numTrees: 10
}, {
	rfr_d9118a3f983e-maxDepth: 10,
	rfr_d9118a3f983e-numTrees: 1
}, {
	rfr_d9118a3f983e-maxDepth: 10,
	rfr_d9118a3f983e-numTrees: 5
}, {
	rfr_d9118a3f983e-maxDepth: 10,
	rfr_d9118a3f983e-numTrees: 10
}, {
	rfr_d9118a3f983e-maxDepth: 15,
	rfr_d9118a3f983e-numTrees: 1
}, {
	rfr_d9118a3f983e-maxDepth: 15,
	rfr_d9118a3f983e-numTrees: 5
}, {
	rfr_d9118a3...

### 3.3 Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).


🥉 **3. Decision Tree**

For the decision tree model we tuned a ParamGrid to cross validate the different hyperparameters used. We tuned maxBins and maxDepth. 
The areaUnderROC is approximately 0.64 and therefore is the weakest model of the 3 we are evaluating.

Generally decision trees are used because they are simple and easy to understand while at the same time fast for inference. On the other hand, they tend to overfit and it is very likely to find local optimal trees rather than global ones. For these reasons the model is performing poorly compared to other methods. 

🥈 **2. Logistics Regression**

We manually tuned some hyperparameters like elasticNetParam, regParam, and maxIter but the default values perform the best.
The areaUnderROC is approximately 0.65 and it is slightly better than the Decision Tree model. This is likely to happen because logistic regression models in general don't perform well when the feature space is too large or when there's a high number of categorical features.

🥇 **1. Random Forest**

We performed cross-validation over the Random Forest, hence tuning the hyperparameters numTree and maxDepth.
The areaUnderROC is approximately 0.77 and therefore is the strongest model of the 3 we are evaluating. This is because random forests generally present very good predictive performances and they can provide reliable feature importance estimate. On the other hand, training might require a high computational cost and in general predictions are slower.


## 4. What more would you do with this data? Anything to help you devise a better solution?

As the data present not-linearity, the model trained above are not always able to generalise enough to predict the correct label with a confidence higher than 77%. We suggest to use more complex models that are able to capture the non-linearity better such as Neural Networks. As an example we could propose a neural network composed of 3 hidden layers.